Process a single recording. This involves:

- Define a configuration
- Read in the time data
- Preprocess the time data
- Decimate the time data
- Window the decimated data
- Calculate the spectra
- Calculate spectral values at evaluation frequencies
- Do the regression

In [37]:
from pathlib import Path
from resistics.config import Configuration
from resistics.time import TimeReaderNumpy
from resistics.gather import QuickGather

In [38]:
config = Configuration(name="testing")

In [39]:
dir_path = Path("..", "data", "time", "quick", "kap123")
time_data = TimeReaderNumpy().run(dir_path)
ref_time = time_data.metadata.first_time

2023-04-28 08:12:05.333 | INFO     | resistics.time:run:777 - Reading time series data from ../data/time/quick/kap123
2023-04-28 08:12:05.334 | DEBUG    | resistics.time:run:779 - Reading time series metadata from ../data/time/quick/kap123
2023-04-28 08:12:05.335 | DEBUG    | resistics.time:run:787 - Reading samples from 0 to 361511
2023-04-28 08:12:05.337 | INFO     | resistics.time:read_data:1133 - Data successfully read from ../data/time/quick/kap123
2023-04-28 08:12:05.338 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:12:05.338 | DEBUG    | resistics.time:run:790 - Applying scaling to time series data


In [40]:
for processor in config.time_processors:
    time_data = processor.run(time_data)

2023-04-28 08:12:05.362 | INFO     | resistics.time:run:1532 - Removing nan values from channels ['Hx', 'Hy', 'Hz', 'Ex', 'Ey']
2023-04-28 08:12:05.375 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:12:05.376 | INFO     | resistics.time:run:1607 - Removing mean from channels ['Hx', 'Hy', 'Hz', 'Ex', 'Ey']
2023-04-28 08:12:05.383 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32


In [41]:
dec_params = config.dec_setup.run(time_data.metadata.fs)
dec_data = config.decimator.run(dec_params, time_data)

2023-04-28 08:12:05.444 | INFO     | resistics.decimate:run:759 - Decimating level 0 with factor 1
2023-04-28 08:12:05.445 | INFO     | resistics.decimate:run:759 - Decimating level 1 with factor 4
2023-04-28 08:12:05.445 | INFO     | resistics.time:run:2146 - Resampling data from 0.2 Hz to 0.05 Hz
2023-04-28 08:12:05.479 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:12:05.480 | INFO     | resistics.decimate:run:759 - Decimating level 2 with factor 8
2023-04-28 08:12:05.481 | INFO     | resistics.time:run:2146 - Resampling data from 0.05 Hz to 0.00625 Hz
2023-04-28 08:12:05.487 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:12:05.488 | INFO     | resistics.decimate:run:759 - Decimating level 3 with factor 4
2023-04-28 08:12:05.488 | INFO     | resistics.time:run:2146 - Resampling data from 0.00625 Hz to 0.0015625 Hz
2023-04-28 08:12:05.490 | DEBUG    | resistics.time:__init__:479 - Cr

In [42]:
win_params = config.win_setup.run(dec_data.metadata.n_levels, dec_data.metadata.fs)
win_data = config.windower.run(ref_time, win_params, dec_data)

2023-04-28 08:12:05.520 | INFO     | resistics.window:run:1330 - Windowing decimation level 0
2023-04-28 08:12:05.522 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-04-28 08:12:05.525 | INFO     | resistics.window:run:1336 - 3765 windows, size 128, overlap 32
2023-04-28 08:12:05.526 | INFO     | resistics.window:run:1330 - Windowing decimation level 1
2023-04-28 08:12:05.527 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-04-28 08:12:05.529 | INFO     | resistics.window:run:1336 - 941 windows, size 128, overlap 32
2023-04-28 08:12:05.530 | INFO     | resistics.window:run:1330 - Windowing decimation level 2
2023-04-28 08:12:05.531 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-04-28 08:12:05.533 | INFO     | resistics.window:run:1336 - 117 windows, size 128, overlap 32
2023-04-28 08:12:05.535 | INFO     | resistics.window:run:1330 - Windowing 

In [43]:
spec_data = config.fourier.run(win_data)
eval_data = config.evals.run(dec_params, spec_data)

2023-04-28 08:12:05.566 | INFO     | resistics.spectra:run:487 - Performing fourier transforms of windowed decimated data
2023-04-28 08:12:05.566 | INFO     | resistics.spectra:run:489 - Transforming level 0
2023-04-28 08:12:05.567 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:12:05.603 | INFO     | resistics.spectra:run:489 - Transforming level 1
2023-04-28 08:12:05.615 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:12:05.622 | INFO     | resistics.spectra:run:489 - Transforming level 2
2023-04-28 08:12:05.622 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:12:05.624 | INFO     | resistics.spectra:run:489 - Transforming level 3
2023-04-28 08:12:05.625 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:12:05.626 | INFO     | resistics.spectra:run:504 - Fourier transforms completed
2023-04-28 08:12:05.627 | DE

In [44]:
gathered_data = QuickGather().run(dir_path, dec_params, config.tf, eval_data)
reg_data = config.regression_preparer.run(config.tf, gathered_data)

2023-04-28 08:12:05.666 | INFO     | resistics.gather:run:847 - Quick gathering data for regression prepartion
2023-04-28 08:12:05.678 | INFO     | resistics.regression:run:388 - Preparing regression data
2023-04-28 08:12:05.680 | INFO     | resistics.regression:run:389 - Out chans site: kap123
2023-04-28 08:12:05.681 | INFO     | resistics.regression:run:390 - Out chans: ['Ex', 'Ey']
2023-04-28 08:12:05.682 | INFO     | resistics.regression:run:391 - In chans site: kap123
2023-04-28 08:12:05.682 | INFO     | resistics.regression:run:392 - In chans: ['Hx', 'Hy']
2023-04-28 08:12:05.683 | INFO     | resistics.regression:run:393 - Cross chans site: kap123
2023-04-28 08:12:05.683 | INFO     | resistics.regression:run:394 - Cross chans: ['Hx', 'Hy']
2023-04-28 08:12:05.683 | INFO     | resistics.regression:_get_regression_data:419 - Preparing regression data for 20 frequencies
100%|██████████| 20/20 [00:00<00:00, 188.79it/s]


In [45]:
soln = config.solver.run(reg_data)
fig = soln.tf.plot(
    soln.freqs,
    soln.components,
    to_plot=["ExHy", "EyHx"],
    x_lim=[1, 5],
    res_lim=[0, 4],
    legend="128",
    symbol="circle",
)
fig.update_layout(height=900)
fig.show()

2023-04-28 08:12:05.817 | INFO     | resistics.regression:_solve:782 - Solving for 20 evaluation frequencies
100%|██████████| 20/20 [00:04<00:00,  4.47it/s]
